In [ ]:
from IPython.display import Audio
from transformers import pipeline
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google import genai
from google.genai import types

f = open("keys/.gemini.txt")
key = f.read()

client = genai.Client(api_key=key)

with open('images/image_1.jpg', 'rb') as f:
  image_bytes = f.read()

context = [
    types.Part.from_bytes(
        data=image_bytes,
        mime_type='image/jpeg',
    ),
]

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=context,
)

print(response.text)

# Split text into chunks
rt = RecursiveCharacterTextSplitter(separators='\n', chunk_size=100, chunk_overlap=0)
docs = rt.split_text(response.text)

# Initialize the TTS pipeline
synthesizer =pipeline(    task="text-to-speech",    model="suno/bark-small",)
# Generate speech from text chunks

audio_all = np.array([])
for i in docs:
     speech = synthesizer(i)
     audio_all = np.concatenate((audio_all, speech['audio']), axis=None)
# Play the audio
print(speech["sampling_rate"])
Audio(audio_all,  rate=24000)